In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("We got a GPU")
    print(physical_devices[0])
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("Sorry, no GPU for you...")

import sys
import os.path
from os import path
import math
import struct
from struct import unpack
from struct import pack
import datetime

import numpy as np
import pandas as pd

from numba import cuda
from tsnecuda import TSNE
from umap import UMAP
from sklearn import cluster
from sklearn.manifold import TSNE as TSNE_ORI
from sklearn.preprocessing import PowerTransformer, normalize, MinMaxScaler, StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn_extra.cluster import KMedoids

import cudf
import cuml
from cuml.manifold import TSNE as TSNE_cuml
from cuml.manifold import UMAP as UMAP_cuml

import seaborn as sns
from matplotlib import colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import sys; sys.path.append('../FIt-SNE-master/')
from fast_tsne import fast_tsne

2024-07-06 18:06:18.305760: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-06 18:06:18.326238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-06 18:06:18.326260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-06 18:06:18.326893: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-06 18:06:18.330616: I tensorflow/core/platform/cpu_feature_guar

KeyboardInterrupt: 

In [ ]:
# conda activate py39
def isEdge(c_dim, dim, edgeRange=1) :
    flag = (c_dim[0]-edgeRange<0 or c_dim[1]-edgeRange<0 or c_dim[2]-edgeRange<0) or \
            (c_dim[0]+edgeRange>=dim[0] or c_dim[1]+edgeRange>=dim[1] or c_dim[2]+edgeRange>=dim[2])
    return flag

def checkEdge(target, dimidx):
    return True if target<0 or target>=dimidx else False

def getValNearst3axis(c_dim, dim, data):    
    r = 0 if (checkEdge(c_dim[0]+1, dim[0])) else data[c_dim[2]*dim[1]*dim[0]+c_dim[1]*dim[0]+(c_dim[0]+1)][0]
    l = 0 if (checkEdge(c_dim[0]-1, dim[0])) else data[c_dim[2]*dim[1]*dim[0]+c_dim[1]*dim[0]+(c_dim[0]-1)][0]
    d = 0 if (checkEdge(c_dim[1]+1, dim[1])) else data[c_dim[2]*dim[1]*dim[0]+(c_dim[1]+1)*dim[0]+c_dim[0]][0]
    u = 0 if (checkEdge(c_dim[1]-1, dim[1])) else data[c_dim[2]*dim[1]*dim[0]+(c_dim[1]-1)*dim[0]+c_dim[0]][0]
    f = 0 if (checkEdge(c_dim[2]-1, dim[2])) else data[(c_dim[2]-1)*dim[1]*dim[0]+c_dim[1]*dim[0]+c_dim[0]][0]
    b = 0 if (checkEdge(c_dim[2]+1, dim[2])) else data[(c_dim[2]+1)*dim[1]*dim[0]+c_dim[1]*dim[0]+c_dim[0]][0]
    return r, l, u, d, f, b #right, left, up, down, front, back

def getGradMagNearst3axis(c_dim, dim, data):    
    r = 0 if (checkEdge(c_dim[0]+1, dim[0])) else data[c_dim[2]*dim[1]*dim[0]+c_dim[1]*dim[0]+(c_dim[0]+1)][1]
    l = 0 if (checkEdge(c_dim[0]-1, dim[0])) else data[c_dim[2]*dim[1]*dim[0]+c_dim[1]*dim[0]+(c_dim[0]-1)][1]
    d = 0 if (checkEdge(c_dim[1]+1, dim[1])) else data[c_dim[2]*dim[1]*dim[0]+(c_dim[1]+1)*dim[0]+c_dim[0]][1]
    u = 0 if (checkEdge(c_dim[1]-1, dim[1])) else data[c_dim[2]*dim[1]*dim[0]+(c_dim[1]-1)*dim[0]+c_dim[0]][1]
    f = 0 if (checkEdge(c_dim[2]-1, dim[2])) else data[(c_dim[2]-1)*dim[1]*dim[0]+c_dim[1]*dim[0]+c_dim[0]][1]
    b = 0 if (checkEdge(c_dim[2]+1, dim[2])) else data[(c_dim[2]+1)*dim[1]*dim[0]+c_dim[1]*dim[0]+c_dim[0]][1]
    return r, l, u, d, f, b #right, left, up, down, front, back

def printNearstMaxGrad(data, idd, idh, idw):
    ret = []
    arr = [[idd, idh, idw],[idd-1, idh, idw],[idd+1, idh, idw],[idd, idh-1, idw],[idd, idh+1, idw],[idd, idh, idw-1], [idd, idh, idw+1]]
    for _ in arr :
        if _[0]<0 or _[0]>=d :
            ret.append(0.0)
            continue
        if _[1]<0 or _[1]>=h :
            ret.append(0.0)
            continue
        if _[2]<0 or _[2]>=w :
            ret.append(0.0)
            continue
        
        idx = _[0]*w*h + _[1]*w + _[2]
        ret.append(data[idx][0])
    max_value = max(ret)
    max_index = ret.index(max_value)
    if max_index == 0 or max_value == 0.0:
        return arr[max_index], max_value
    else :
        return printNearstMaxGrad(data, arr[max_index][0], arr[max_index][1],arr[max_index][2])
    
def printNearstMinGrad(data, idd, idh, idw):
    ret = []
    arr = [[idd, idh, idw],[idd-1, idh, idw],[idd+1, idh, idw],[idd, idh-1, idw],[idd, idh+1, idw],[idd, idh, idw-1], [idd, idh, idw+1]]
    for _ in arr :
        if _[0]<0 or _[0]>=d :
            ret.append(1000000000.0)
            continue
        if _[1]<0 or _[1]>=h :
            ret.append(1000000000.0)
            continue
        if _[2]<0 or _[2]>=w :
            ret.append(1000000000.0)
            continue
        idx = _[0]*w*h + _[1]*w + _[2]
        ret.append(data[idx][0])
    min_value = min(ret)
    min_index = ret.index(min_value)
    if min_index == 0 or min_value == 0.0:
        return arr[min_index], min_value
    else :
        return printNearstMinGrad(data, arr[min_index][0], arr[min_index][1],arr[min_index][2])


def printNearstVal(data, idd, idh, idw):
    maxIdx, maxval = printNearstMaxGrad(data, idd, idh, idw)       
    minIdx, minval = printNearstMinGrad(data, idd, idh, idw)       
    
    return [minval,maxval]

def _TSNE(learning_rate, _perplexity, data, _random_state = 0) :
    #model = TSNE(random_state = _random_state, n_components=2, perplexity=_perplexity, n_iter=learning_rate)
    model = TSNE(n_components=2, perplexity=_perplexity, learning_rate=learning_rate)
    print("TSNE with CUDA calc : ", end='')
    startTime = datetime.datetime.now()
    transformed = model.fit_transform(data)
    endTime = datetime.datetime.now()
    diffTime = endTime-startTime
    exeTime = diffTime.total_seconds() * 1000
    print(exeTime,'ms')
    return transformed

def _TSNE_cuml(learning_rate, _perplexity, data, _random_state = 0) :
    #model = TSNE(random_state = _random_state, n_components=2, perplexity=_perplexity, n_iter=learning_rate)
    model = TSNE_cuml(n_components=2, perplexity=_perplexity, learning_rate=learning_rate)
    print("TSNE with CUDA (cuML) calc : ", end='')
    startTime = datetime.datetime.now()
    transformed = model.fit_transform(data)
    endTime = datetime.datetime.now()
    diffTime = endTime-startTime
    exeTime = diffTime.total_seconds() * 1000
    print(exeTime,'ms')
    return transformed

def _TSNE_ORI(data, learning_rate, _perplexity, _random_state = 0) :
    #model = TSNE(random_state = _random_state, n_components=2, perplexity=_perplexity, n_iter=learning_rate)
    model = TSNE_ORI(random_state = _random_state, n_components=2, perplexity=_perplexity, n_iter=learning_rate)
    print("TSNE calc : ", end='')
    startTime = datetime.datetime.now()
    transformed = model.fit_transform(data)
    endTime = datetime.datetime.now()
    diffTime = endTime-startTime
    exeTime = diffTime.total_seconds() * 1000
    print(exeTime,'ms')
    return transformed

def _FITSNE(data, _random_state = 0):
    print("FITSNE calc : ", end='')
    startTime = datetime.datetime.now()
    fast_tsne(data, seed=0)
    endTime = datetime.datetime.now()
    diffTime = endTime-startTime
    exeTime = diffTime.total_seconds() * 1000
    print(exeTime,'ms')

def drawDataHistogram_custom_2by4(data, title) :
    fig, ((ax1, ax2, ax3,ax4,ax5),(ax6,ax7,ax8,ax9,ax10),(ax11,ax12,ax13,ax14,ax15)) = plt.subplots(3, 5)
    fig.set_size_inches(18, 7)
    fig.suptitle(title, fontsize=12)
    ax1.set_title("i");         ax2.set_title("j");       ax3.set_title("k");     ax4.set_title("Value");   ax5.set_title("GradMag")
    ax6.set_title("rightVal");  ax7.set_title("leftVal"); ax8.set_title("upVal"); ax9.set_title("DownVal"); ax10.set_title("frontVal")
    ax11.set_title("backVal");  ax12.set_title("LH_L");   ax13.set_title("LH_H"); ax14.set_title("none");   ax15.set_title("none")

    sns.distplot(data[:,0], color="green", ax=ax1)
    sns.distplot(data[:,1], color="green", ax=ax2)
    sns.distplot(data[:,2], color="green", ax=ax3)
    sns.distplot(data[:,3], color="green", ax=ax4)
    sns.distplot(data[:,4], color="green", ax=ax5)
    sns.distplot(data[:,5], color="green", ax=ax6)
    sns.distplot(data[:,6], color="green", ax=ax7)
    sns.distplot(data[:,7], color="green", ax=ax8)
    sns.distplot(data[:,8], color="green", ax=ax9)
    sns.distplot(data[:,9], color="green", ax=ax10)
    sns.distplot(data[:,10], color="green", ax=ax11)
    sns.distplot(data[:,11], color="green", ax=ax12)
    sns.distplot(data[:,12], color="green", ax=ax13)

def rangecheck(tx, ty, xmin, ymin, xmax, ymax):
    if tx<xmin or tx>xmax or ty<ymin or ty>ymax:
        return False
    else :
        return True

print('[Success]program functions load complete!')

In [ ]:
fname = 'Carp_256x256x512.raw'
print(fname)

w = int(fname.split('_')[1].split('.')[0].split('x')[0] )
h = int(fname.split('_')[1].split('.')[0].split('x')[1] )
d = int(fname.split('_')[1].split('.')[0].split('x')[2] )
size = w*h*d

if path.exists(fname) != True:
    print("No file")

voldata = []
with open('./%s.Float.TextureCache'%(fname), 'rb') as fp:
    data_min = unpack('<f', fp.read(4))[0]
    data_max = unpack('<f', fp.read(4))[0]
    grad_min = unpack('<f', fp.read(4))[0]
    grad_max = unpack('<f', fp.read(4))[0]
    
    for k in range(d) :
        for j in range(h):
            for i in range(w):
                gradx = unpack('<f', fp.read(4))[0]
                grady = unpack('<f', fp.read(4))[0]
                gradz = unpack('<f', fp.read(4))[0]
                gradmag = math.sqrt(gradx*gradx+grady*grady+gradz*gradz)
                val = unpack('<f', fp.read(4))[0]
                voldata.append([val, gradmag])
                
np_voldata = np.array(voldata)

LHarray = [[] for i in range(d*h*w)]
print("len(LHarray): "+str(len(LHarray)))

for dd in range(d):
    for hh in range(h):
        for ww in range(w):
            LHarray[dd*w*h + hh*w + ww] = printNearstVal(np_voldata, dd,hh,ww)
tt = np.array(LHarray)

md_sparse_D19=[]
boxes = [[[43, 401],[105,475]],[[280, 398],[309,480]],[[309,341],[417,480]],]
normBoxes = [[[0.0,0.0],[0.0,0.0]],[[0.0,0.0],[0.0,0.0]],[[0.0,0.0],[0.0,0.0]],]

xrange = [0,512]; yrange = [33,480]
for bidx in range(len(boxes)):
    normBoxes[bidx][0][0] = (boxes[bidx][0][0]-xrange[0])/(xrange[1]-xrange[0])
    normBoxes[bidx][1][0] = (boxes[bidx][1][0]-xrange[0])/(xrange[1]-xrange[0])
    normBoxes[bidx][0][1] = 1.0-(boxes[bidx][1][1]-yrange[0])/(yrange[1]-yrange[0]) 
    normBoxes[bidx][1][1] = 1.0-(boxes[bidx][0][1]-yrange[0])/(yrange[1]-yrange[0]) 

vecLength = math.sqrt(w*w+h*h+d*d)

for k in range(d):
    for j in range(h):
        for i in range(w):
            c_idx = k*w*h + j*w + i
            r,l,u,d,f,b = getValNearst3axis([i,j,k], [w,h,d], np_voldata)
            gr,gl,gu,gd,gf,gb = getGradMagNearst3axis([i,j,k], [w,h,d], np_voldata)
            
            if np_voldata[c_idx][0] == 0 :
                continue
            else :
                for bidx in range(len(normBoxes)):
                    if (rangecheck(np_voldata[c_idx][0],np_voldata[c_idx][1],normBoxes[bidx][0][0], normBoxes[bidx][0][1], normBoxes[bidx][1][0], normBoxes[bidx][1][1]))==True:
                        md_sparse_D19.append([i,j,k,np_voldata[c_idx][0],np_voldata[c_idx][1], r, l, u, d, f, b, gr,gl,gu,gd,gf,gb,LHarray[c_idx][0], LHarray[c_idx][1]])
np_md_sparse_D19 = np.array(md_sparse_D19); print(len(md_sparse_D19))